<a href="https://colab.research.google.com/github/Saikiran4545/testing1/blob/master/Trading_Bot1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ccxt
import time

class SimpleTradingBot:
    def __init__(self, initial_budget, symbol, api_key, api_secret, api_passphrase):
        self.budget = initial_budget
        self.symbol = symbol
        self.exchange = ccxt.kucoin({
            'apiKey': api_key,
            'secret': api_secret,
            'password': api_passphrase,
        })

    def place_order(self, order_type, price, quantity):
        try:
            order = None
            if order_type.lower() == "buy":
                order = self.exchange.create_limit_buy_order(self.symbol, quantity, price)
            elif order_type.lower() == "sell":
                order = self.exchange.create_limit_sell_order(self.symbol, quantity, price)

            order_id = order['id'] if order else None
            print(f"{order_type} order placed - Order ID: {order_id}")
            return order_id
        except Exception as e:
            print(f"Error during order placement: {e}")
            return None

    def execute_strategy(self):
        try:
            ticker = self.exchange.fetch_ticker(self.symbol)
            current_price = ticker['ask']

            if self.budget > 0:
                # Place a buy order
                buy_price = current_price
                buy_quantity = min(self.budget / buy_price, 1.0)
                buy_order_id = self.place_order("buy", buy_price, buy_quantity)

                if buy_order_id:
                    self.budget -= buy_price * buy_quantity
                    print(f"Budget after buying: {self.budget:.2f}")

                    # Place a corresponding sell order with a profit target of 0.05%
                    sell_price = buy_price * 1.0005
                    sell_quantity = buy_quantity
                    sell_order_id = self.place_order("sell", sell_price, sell_quantity)

                    if sell_order_id:
                        print("Sell order placed.")
                        return True

        except Exception as e:
            print(f"Error in executing strategy: {e}")

        return False

    def run(self):
        while True:
            strategy_executed = self.execute_strategy()

            if strategy_executed:
                # Wait for the last sell order to execute
                print("Waiting for the last sell order to execute...")
                while True:
                    try:
                        order_status = self.exchange.fetch_order(self.exchange.fetch_open_orders(self.symbol)[0]['id'])
                        if order_status['status'] == 'closed':
                            print("Last sell order executed. Placing new buy and sell orders.")
                            break
                        time.sleep(10)
                    except Exception as e:
                        print(f"Error checking last sell order: {e}")

            # Introduce a delay between iterations
            time.sleep(60)  # 60 seconds interval between iterations

if __name__ == "__main__":
    initial_budget = 50
    symbol = 'SOL/USDT'
    api_key = '657d43f44d8dba0001eb4c47'
    api_secret = '8d32247d-3752-49f4-a76e-c69f71f20cf3'
    api_passphrase = 'Mahadev@4545'

    trading_bot = SimpleTradingBot(initial_budget, symbol, api_key, api_secret, api_passphrase)

    try:
        trading_bot.run()
    except KeyboardInterrupt:
        print("Exiting...")
